<a href="https://colab.research.google.com/github/fatemafaria142/Evaluating-LLMs-Generated-Food-Recipes-in-Comparison-to-Established-Reference-Recipes/blob/main/Recipe_Generation_using_Starling_LM_7B_alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


#### **Dataset Link:** https://huggingface.co/datasets/recipe_nlg

In [3]:
import pandas as pd

# Load the original dataset
dataset = pd.read_csv("/content/drive/MyDrive/Recipe Generation/full_dataset.csv")

# Extract the first 3000 rows
subset_dataset = dataset.head(3000)

# Save the subset to a new CSV file
subset_dataset.to_csv("/content/drive/MyDrive/Recipe Generation/recipe_generation_subset.csv", index=False)


In [4]:
import pandas as pd
instruct_tune_dataset = pd.read_csv("/content/drive/MyDrive/Recipe Generation/recipe_generation_subset.csv")
instruct_tune_dataset.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [5]:
length_of_data = len(instruct_tune_dataset)
# Print the length
print(length_of_data)

3000


In [6]:
from datasets import Dataset, DatasetDict

# Create a DatasetDict for instruct_tune_dataset
instruct_tune_dataset = DatasetDict()

# Select 1500 rows for the 'train' split and only keep 'ingredients' and 'directions'
instruct_tune_dataset["train"] = Dataset.from_pandas(dataset[["ingredients", "directions"]].head(2500))

# Select 200 rows for the 'test' split and only keep 'ingredients' and 'directions'
instruct_tune_dataset["test"] = Dataset.from_pandas(dataset[["ingredients", "directions"]].tail(200))

# Print the updated instruct_tune_dataset
print(instruct_tune_dataset)

DatasetDict({
    train: Dataset({
        features: ['ingredients', 'directions'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['ingredients', 'directions'],
        num_rows: 200
    })
})


### **We will use just a small subset of the data for this training example**

In [7]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['ingredients', 'directions'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['ingredients', 'directions'],
        num_rows: 200
    })
})

In [11]:
def create_prompt(sample):
    # Use a predefined template for instructions
    prompt = "Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: "
    prompt += sample['ingredients']
    single_turn_prompt = f"GPT4 Correct User: {prompt}<|end_of_turn|> GPT4 Correct Assistant: {sample['directions']}"
    return single_turn_prompt

In [12]:
create_prompt(instruct_tune_dataset["train"][0])

'GPT4 Correct User: Imagine you\'re a chef, and you\'ve got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]<|end_of_turn|> GPT4 Correct Assistant: ["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]'

In [13]:
create_prompt(instruct_tune_dataset["train"][1])

'GPT4 Correct User: Imagine you\'re a chef, and you\'ve got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]<|end_of_turn|> GPT4 Correct Assistant: ["Place chipped beef on bottom of baking dish.", "Place chicken on top of beef.", "Mix soup and cream together; pour over chicken. Bake, uncovered, at 275\\u00b0 for 3 hours."]'

In [14]:
create_prompt(instruct_tune_dataset["train"][2])

'GPT4 Correct User: Imagine you\'re a chef, and you\'ve got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg. cream cheese, cubed", "1/3 c. butter, cubed", "1/2 tsp. garlic powder", "1/2 tsp. salt", "1/4 tsp. pepper"]<|end_of_turn|> GPT4 Correct Assistant: ["In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings."]'

In [15]:
create_prompt(instruct_tune_dataset["train"][3])

'GPT4 Correct User: Imagine you\'re a chef, and you\'ve got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["1 large whole chicken", "2 (10 1/2 oz.) cans chicken gravy", "1 (10 1/2 oz.) can cream of mushroom soup", "1 (6 oz.) box Stove Top stuffing", "4 oz. shredded cheese"]<|end_of_turn|> GPT4 Correct Assistant: ["Boil and debone chicken.", "Put bite size pieces in average size square casserole dish.", "Pour gravy and cream of mushroom soup over chicken; level.", "Make stuffing according to instructions on box (do not make too moist).", "Put stuffing on top of chicken and gravy; level.", "Sprinkle shredded cheese on top and bake at 350\\u00b0 for approximately 20 minutes or until golden and bubbly."]'

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* **Model and Tokenizer:** https://huggingface.co/berkeley-nest/Starling-LM-7B-alpha

In [19]:
mode_id = "berkeley-nest/Starling-LM-7B-alpha"

In [20]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [21]:
tokenizer = AutoTokenizer.from_pretrained("berkeley-nest/Starling-LM-7B-alpha")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **Let's example how well the model does at this task currently:**

In [22]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1024, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [24]:
prompt = """GPT4 Correct User: Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["6 baking potatoes", "1 lb. of extra lean ground beef", "2/3 c. butter or margarine", "6 c. milk", "3/4 tsp. salt", "1/2 tsp. pepper", "1 1/2 c (6 oz.) shredded Cheddar cheese, divided", "12 sliced bacon, cooked, crumbled and divided", "4 green onion, chopped and divided", "1 (8 oz.) carton sour cream (optional)"] <|end_of_turn|> GPT4 Correct Assistant:"""

In [25]:
generate_response(prompt, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'<s> GPT4 Correct User: Imagine you\'re a chef, and you\'ve got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["6 baking potatoes", "1 lb. of extra lean ground beef", "2/3 c. butter or margarine", "6 c. milk", "3/4 tsp. salt", "1/2 tsp. pepper", "1 1/2 c (6 oz.) shredded Cheddar cheese, divided", "12 sliced bacon, cooked, crumbled and divided", "4 green onion, chopped and divided", "1 (8 oz.) carton sour cream (optional)"] <|end_of_turn|>  GPT4 Correct Assistant: Title: Loaded Potato Bake with Beef and Bacon\n\nIngredients:\n- 6 baking potatoes\n- 1 lb. of extra lean ground beef\n- 2/3 c. butter or margarine\n- 6 c. milk\n- 3/4 tsp. salt\n- 1/2 tsp. pepper\n- 1 1/2 c (6 oz.) shredded Cheddar cheese, divided\n- 12 sliced bacon, cooked, crumbled and divided\n- 4 green onion, chopped and divided\n- 1 (8 oz.) carton sour cream (optional)\n\nDirections:\n\n1. Preheat the oven

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [26]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [27]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [28]:
from transformers import TrainingArguments
output_model= "Starling_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=150,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [31]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

### **Training starts here**

In [32]:
trainer.train()

Step,Training Loss
10,1.192000
20,0.899200
30,0.823000
40,0.820400
50,0.833100
60,0.843100
70,0.805200
80,0.829900
90,0.779200
100,0.820900


TrainOutput(global_step=150, training_loss=0.8449023755391439, metrics={'train_runtime': 1233.2531, 'train_samples_per_second': 0.487, 'train_steps_per_second': 0.122, 'total_flos': 1.3112665178112e+16, 'train_loss': 0.8449023755391439, 'epoch': 0.57})

### **Save the model**

In [33]:
trainer.save_model("Starling_instruct_generation")

In [34]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [35]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=512, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

### **Example No:1**

In [38]:
prompt = "GPT4 Correct User: Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: "
prompt += '''["6 baking potatoes", "1 lb. of extra lean ground beef", "2/3 c. butter or margarine", "6 c. milk", "3/4 tsp. salt", "1/2 tsp. pepper", "1 1/2 c (6 oz.) shredded Cheddar cheese, divided", "12 sliced bacon, cooked, crumbled and divided", "4 green onion, chopped and divided", "1 (8 oz.) carton sour cream (optional)"]<|end_of_turn|> '''
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s> GPT4 Correct User: Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["6 baking potatoes", "1 lb. of extra lean ground beef", "2/3 c. butter or margarine", "6 c. milk", "3/4 tsp. salt", "1/2 tsp. pepper", "1 1/2 c (6 oz.) shredded Cheddar cheese, divided", "12 sliced bacon, cooked, crumbled and divided", "4 green onion, chopped and divided", "1 (8 oz.) carton sour cream (optional)"]<|end_of_turn|>  GPT4 Correct Assistant: 1. Preheat oven to 400° F. 2. Scrub potatoes; peel if desired. Bake potatoes until tender (about 1 to 1 1/2 hours). 3. Meanwhile, in large skillet, cook ground beef, onion, salt and pepper until brown and thoroughly cooked. Drain fat; add 1/2 cup of the milk to beef mixture and cook 2 to 3 minutes on low heat, stirring occasionally. 4. When potatoes are cool, peel and mash or chop, depending on desired consistency. 

### **Example No:2**

In [39]:
prompt = "GPT4 Correct User: Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: "
prompt += '''["1 1/2 c. sugar", "1/2 c. butter", "1 egg", "1 c. buttermilk", "2 c. flour", "1/2 tsp. salt", "1 tsp. soda", "1 c. buttermilk", "2 c. rhubarb, finely cut", "1 tsp. vanilla"]<|end_of_turn|> '''
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> GPT4 Correct User: Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["1 1/2 c. sugar", "1/2 c. butter", "1 egg", "1 c. buttermilk", "2 c. flour", "1/2 tsp. salt", "1 tsp. soda", "1 c. buttermilk", "2 c. rhubarb, finely cut", "1 tsp. vanilla"]<|end_of_turn|>  GPT4 Correct Assistant: Preheat oven to 350 degrees. Cream together sugar, butter and egg. Add flour, salt, soda and buttermilk; mix well. Add the second cup of buttermilk, rhubarb and vanilla; mix until well blended. Pour into a greased and floured 9 x 13-inch pan. Bake for 45 to 50 minutes or until a tester inserted in the center comes out clean.<|end_of_turn|>


### **Example No:3**

In [40]:
prompt = "GPT4 Correct User: Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: "
prompt += '''["1 can cream-style corn", "1 can whole kernel corn", "1/2 pkg. (approximately 20) saltine crackers, crushed", "1 egg, beaten", "6 tsp. butter, divided", "pepper to taste"] <|end_of_turn|> '''
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> GPT4 Correct User: Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["1 can cream-style corn", "1 can whole kernel corn", "1/2 pkg. (approximately 20) saltine crackers, crushed", "1 egg, beaten", "6 tsp. butter, divided", "pepper to taste"] <|end_of_turn|>  GPT4 Correct Assistant: First, preheat the oven to 350 degrees F (175 degrees C).  Next, in a large bowl, combine the cream-style corn, whole kernel corn, crushed saltines and beaten egg.  Mix well.  In a 2-quart baking dish, place 3 teaspoons of butter.  Spread mixture over the Butter.  Sprinkle the top with the remaining butter and pepper to taste.  Bake for 30 to 35 minutes, or until bubbly and golden brown.<|end_of_turn|>
